In [1]:
# import BreakfastNaive as BF
import BreakfastNaiveFS as BF
import torch
import torch.nn as nn
from model import NeuralNet
import os
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import importlib
importlib.reload(BF)

<module 'BreakfastNaiveFS' from 'C:\\Users\\dcsang\\PycharmProjects\\feedfwdBF\\BreakfastNaiveFS.py'>

In [3]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(r"C:\Users\dcsang\PycharmProjects\feedfwdBF\logs\fs\runs")

In [ ]:
# for _ in range(10):
#     writer.add_scalar('Accuracy over test set per epoch',
#                       10,1)
# writer.close()

In [4]:
# DATASET for website original (GT from FS though)
# visual_feat_path = r"C:\Users\dcsang\PycharmProjects\embedding\breakfast\bf_kinetics_feat"
# text_path = r"C:\Users\dcsang\PycharmProjects\embedding\breakfast\groundTruth"
# map_path = r"C:\Users\dcsang\PycharmProjects\embedding\breakfast\mapping.txt"

# DATASET for fs dataset
visual_feat_path = r"C:\Users\dcsang\PycharmProjects\embedding\breakfast\Breakfast_fs\data_npy"
text_path = r"C:\Users\dcsang\PycharmProjects\embedding\breakfast\Breakfast_fs\groundTruth"
map_path = r"C:\Users\dcsang\PycharmProjects\embedding\breakfast\Breakfast_fs\splits\mapping.txt"


visual_feat_path_train = os.path.join(visual_feat_path, "train")
text_path_train = os.path.join(text_path, "train")

visual_feat_path_test = os.path.join(visual_feat_path, "test")
text_path_test = os.path.join(text_path, "test")
# train_dataset = BF.BreakfastNaive(visual_feat_path_train, text_path_train, map_path)
# test_dataset = BF.BreakfastNaive(visual_feat_path_test, text_path_test, map_path)

train_dataset = BF.BreakfastNaiveFS(visual_feat_path_train, text_path_train, map_path)
test_dataset = BF.BreakfastNaiveFS(visual_feat_path_test, text_path_test, map_path)

In [5]:
# Hyper-parameters
input_size = 400
hidden_size = 128
num_classes = 48
num_epochs = 100  # TODO
batch_size = 100  # TODO increase
learning_rate = 0.00001  # TODO

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
def visualise(accu,loss, epoch):
    writer.add_scalar('Accuracy over test set per epoch',
                      accu,epoch)
    #TODO: Add accuracy over train set
#     write.add_scalar(' Accuracy over train set per epoch',
#                     accu_train, epoch)
    writer.add_scalar('training loss per epoch',
                      loss,epoch)

In [8]:
def test_fn():
    # TEST
    with torch.no_grad():
        correct = 0
        total = 0
        for data_batch in test_loader:
            vis_feats = data_batch['vis_feats'].to(device)
            labels = data_batch['labels'].to(device)

            outputs = model(vis_feats.float())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            #TODO print confusion matrix

        print('Accuracy over entire test set: {} %'.format(100 * correct / total))
    accuracy = (100 * correct / total)
    return accuracy

In [9]:


# MODEL
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# TRAIN
total_step = len(train_loader)
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for i, data_batch in enumerate(train_loader):
        vis_feats = data_batch['vis_feats'].to(device)
        labels = data_batch['labels'].to(device)  # TODO dont send one of them to gpu and see what happens?

        # fwd pass
        outputs = model(vis_feats.float())
        loss = criterion(outputs, labels.long())

        # bkwd pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
    epoch_accuracy = test_fn()
    visualise(epoch_accuracy, epoch_loss, epoch+1)


Epoch [1/100], Step [10/99], Loss: 6.6049
Epoch [1/100], Step [20/99], Loss: 6.5110
Epoch [1/100], Step [30/99], Loss: 5.7708
Epoch [1/100], Step [40/99], Loss: 5.5287
Epoch [1/100], Step [50/99], Loss: 5.7466
Epoch [1/100], Step [60/99], Loss: 5.3736
Epoch [1/100], Step [70/99], Loss: 5.0333
Epoch [1/100], Step [80/99], Loss: 4.8913
Epoch [1/100], Step [90/99], Loss: 4.8614
Accuracy over entire test set: 10.954263128176171 %
Epoch [2/100], Step [10/99], Loss: 4.4408
Epoch [2/100], Step [20/99], Loss: 4.5416
Epoch [2/100], Step [30/99], Loss: 4.4748
Epoch [2/100], Step [40/99], Loss: 4.0512
Epoch [2/100], Step [50/99], Loss: 4.0307
Epoch [2/100], Step [60/99], Loss: 3.7145
Epoch [2/100], Step [70/99], Loss: 4.0856
Epoch [2/100], Step [80/99], Loss: 3.8470
Epoch [2/100], Step [90/99], Loss: 4.2271
Accuracy over entire test set: 20.10163749294184 %
Epoch [3/100], Step [10/99], Loss: 3.6601
Epoch [3/100], Step [20/99], Loss: 3.9680
Epoch [3/100], Step [30/99], Loss: 3.9277
Epoch [3/100], 

Accuracy over entire test set: 38.00112930547713 %
Epoch [20/100], Step [10/99], Loss: 2.2644
Epoch [20/100], Step [20/99], Loss: 2.3058
Epoch [20/100], Step [30/99], Loss: 2.3675
Epoch [20/100], Step [40/99], Loss: 2.2142
Epoch [20/100], Step [50/99], Loss: 2.3384
Epoch [20/100], Step [60/99], Loss: 2.3376
Epoch [20/100], Step [70/99], Loss: 2.3410
Epoch [20/100], Step [80/99], Loss: 2.2931
Epoch [20/100], Step [90/99], Loss: 2.2464
Accuracy over entire test set: 38.05759457933371 %
Epoch [21/100], Step [10/99], Loss: 2.3737
Epoch [21/100], Step [20/99], Loss: 2.2042
Epoch [21/100], Step [30/99], Loss: 2.2102
Epoch [21/100], Step [40/99], Loss: 2.4917
Epoch [21/100], Step [50/99], Loss: 2.4594
Epoch [21/100], Step [60/99], Loss: 2.7426
Epoch [21/100], Step [70/99], Loss: 2.3502
Epoch [21/100], Step [80/99], Loss: 2.2967
Epoch [21/100], Step [90/99], Loss: 2.5515
Accuracy over entire test set: 38.73517786561265 %
Epoch [22/100], Step [10/99], Loss: 2.3476
Epoch [22/100], Step [20/99], 

Accuracy over entire test set: 42.51835121400339 %
Epoch [39/100], Step [10/99], Loss: 2.1185
Epoch [39/100], Step [20/99], Loss: 1.9525
Epoch [39/100], Step [30/99], Loss: 1.9943
Epoch [39/100], Step [40/99], Loss: 1.7611
Epoch [39/100], Step [50/99], Loss: 1.9529
Epoch [39/100], Step [60/99], Loss: 1.9230
Epoch [39/100], Step [70/99], Loss: 2.1170
Epoch [39/100], Step [80/99], Loss: 1.8143
Epoch [39/100], Step [90/99], Loss: 2.0587
Accuracy over entire test set: 42.57481648785997 %
Epoch [40/100], Step [10/99], Loss: 1.8376
Epoch [40/100], Step [20/99], Loss: 2.0650
Epoch [40/100], Step [30/99], Loss: 1.9405
Epoch [40/100], Step [40/99], Loss: 1.7608
Epoch [40/100], Step [50/99], Loss: 1.9736
Epoch [40/100], Step [60/99], Loss: 1.7048
Epoch [40/100], Step [70/99], Loss: 1.9281
Epoch [40/100], Step [80/99], Loss: 1.9717
Epoch [40/100], Step [90/99], Loss: 1.9090
Accuracy over entire test set: 43.19593450028233 %
Epoch [41/100], Step [10/99], Loss: 1.9781
Epoch [41/100], Step [20/99], 

Epoch [57/100], Step [70/99], Loss: 1.7905
Epoch [57/100], Step [80/99], Loss: 1.6851
Epoch [57/100], Step [90/99], Loss: 1.7841
Accuracy over entire test set: 45.28514963297572 %
Epoch [58/100], Step [10/99], Loss: 1.9445
Epoch [58/100], Step [20/99], Loss: 1.7428
Epoch [58/100], Step [30/99], Loss: 1.8466
Epoch [58/100], Step [40/99], Loss: 1.6018
Epoch [58/100], Step [50/99], Loss: 2.0095
Epoch [58/100], Step [60/99], Loss: 1.6157
Epoch [58/100], Step [70/99], Loss: 1.7114
Epoch [58/100], Step [80/99], Loss: 1.7735
Epoch [58/100], Step [90/99], Loss: 1.7591
Accuracy over entire test set: 45.39808018068888 %
Epoch [59/100], Step [10/99], Loss: 1.6699
Epoch [59/100], Step [20/99], Loss: 1.8288
Epoch [59/100], Step [30/99], Loss: 1.7888
Epoch [59/100], Step [40/99], Loss: 1.5700
Epoch [59/100], Step [50/99], Loss: 1.8967
Epoch [59/100], Step [60/99], Loss: 1.7248
Epoch [59/100], Step [70/99], Loss: 1.8111
Epoch [59/100], Step [80/99], Loss: 1.8838
Epoch [59/100], Step [90/99], Loss: 1.

Epoch [76/100], Step [30/99], Loss: 1.5144
Epoch [76/100], Step [40/99], Loss: 1.5592
Epoch [76/100], Step [50/99], Loss: 1.3425
Epoch [76/100], Step [60/99], Loss: 1.7109
Epoch [76/100], Step [70/99], Loss: 1.7532
Epoch [76/100], Step [80/99], Loss: 1.6816
Epoch [76/100], Step [90/99], Loss: 1.4650
Accuracy over entire test set: 47.430830039525695 %
Epoch [77/100], Step [10/99], Loss: 1.4722
Epoch [77/100], Step [20/99], Loss: 1.4849
Epoch [77/100], Step [30/99], Loss: 1.5315
Epoch [77/100], Step [40/99], Loss: 1.3416
Epoch [77/100], Step [50/99], Loss: 1.7054
Epoch [77/100], Step [60/99], Loss: 1.3791
Epoch [77/100], Step [70/99], Loss: 1.6715
Epoch [77/100], Step [80/99], Loss: 1.5271
Epoch [77/100], Step [90/99], Loss: 1.3251
Accuracy over entire test set: 47.09203839638622 %
Epoch [78/100], Step [10/99], Loss: 1.6570
Epoch [78/100], Step [20/99], Loss: 1.4762
Epoch [78/100], Step [30/99], Loss: 1.6720
Epoch [78/100], Step [40/99], Loss: 1.5639
Epoch [78/100], Step [50/99], Loss: 1

Epoch [95/100], Step [30/99], Loss: 1.5729
Epoch [95/100], Step [40/99], Loss: 1.6989
Epoch [95/100], Step [50/99], Loss: 1.5844
Epoch [95/100], Step [60/99], Loss: 1.9006
Epoch [95/100], Step [70/99], Loss: 1.7393
Epoch [95/100], Step [80/99], Loss: 1.5206
Epoch [95/100], Step [90/99], Loss: 1.4553
Accuracy over entire test set: 48.560135516657255 %
Epoch [96/100], Step [10/99], Loss: 1.3268
Epoch [96/100], Step [20/99], Loss: 1.5549
Epoch [96/100], Step [30/99], Loss: 1.3930
Epoch [96/100], Step [40/99], Loss: 1.4473
Epoch [96/100], Step [50/99], Loss: 1.3275
Epoch [96/100], Step [60/99], Loss: 1.4097
Epoch [96/100], Step [70/99], Loss: 1.3939
Epoch [96/100], Step [80/99], Loss: 1.4084
Epoch [96/100], Step [90/99], Loss: 1.6765
Accuracy over entire test set: 49.520045172219085 %
Epoch [97/100], Step [10/99], Loss: 1.5449
Epoch [97/100], Step [20/99], Loss: 1.3557
Epoch [97/100], Step [30/99], Loss: 1.3942
Epoch [97/100], Step [40/99], Loss: 1.3422
Epoch [97/100], Step [50/99], Loss: 